In [1]:
#Librerías
import numpy as np

In [2]:
#Forma polinomial
class Polynomial:
    def __init__(self, coefficients,z0):
        
        self.coeffs=coefficients
        self.point= z0
        
    def __str__(self):
        out = ''
        size = len(self.coeffs)
        point = self.point
        
        for i in range(size-1):    
            if self.coeffs[i] != 0:
                out += ' + %s*(z-%s)^%d' % (self.coeffs[i], point,size-i-1) 
                
        out += ' + %s' % (self.coeffs[-1])
        out = out.replace('+ -', '- ')
        out = out.replace('--', '+')
        out = out.replace(' 1*', ' ')
        out = out.replace(' *1', ' ')
        out = out.replace(')^1 ', ') ')
        out = out.replace('(z-0)', 'z')
        out = out.replace('.0*', '*')    
            
        if out[0:3] == ' + ': 
            out = out[3:]
            
        if out[0:3] == ' - ':
            out = '-' + out[3:]
        return out
    
#Forma factorizada
class Factorization:
    def __init__(self, roots, coefficients):  
        self.roots = roots
        self.coeffs = coefficients
    def __str__(self):
        
        if self.coeffs[0] != 0:
            out = '%s' % self.coeffs[0]
            out = out.replace('j)','*i)')
            size = len(self.roots)  
            for i in range(size):
                    out += '(z - %s - %s*i)' % (self.roots[i].real, self.roots[i].imag)        
            out = out.replace('- -', '+ ')
            out = out.replace(' - 0.0*i)', ')')
            out = out.replace(' + 0.0*i)', ')')
            out = out.replace(' - 0.0','')
            out = out.replace('1.0*i','i')
            out = out.replace('1(','(')
        else:
            out = '0'
        return out

In [3]:
#Algoritmo de Laguerre-Horner
def laguerre_horner(coeffs, z0, mydel, max_iter = 200):
    '''p(x) = 0 usando el método de Laguerre
    ----------
    coeffs : vector de coeficientes, z0 : Aproximación inicial, 
    mydel : Tolerancia al incremento, max_iter : Número maximo de iteraciones
    ----------  '''
    
    z_0 = [z0][:]
    z = z0
    for k in range(1,max_iter):
        a = coeffs[0]
        b = 0
        c = 0
        for j in range(1,len(coeffs)):
            c = b +z0*c
            b = a + z0*b
            a = z0*a + coeffs[j]
        c = 2*c
        if a == 0:
            return z
        A = -b/a
        B = A**2-c/a
        n = len(coeffs)-1
        if abs((A+np.sqrt((n-1)*(n*B-A**2)))/n) >= abs((A-np.sqrt((n-1)*(n*B-A**2)))/n):
            C = (A+np.sqrt((n-1)*(n*B-A**2)))/n
        else:
            C = (A-np.sqrt((n-1)*(n*B-A**2)))/n
        z = z+1/C
        #print( 'z(',k,')=', z) #imprimir cada iteración del algoritmo    
        if abs(z-z0) < mydel :
            return z
        z0 = z
    print('Número máximo de iteraciones alcanzado')
    return None

In [4]:
#Deflación de p(z) entre (z-z0)
def deflacion(coeffs,z0):
    for k in range(1,len(coeffs)):
        coeffs[k] = z0*coeffs[k-1] + coeffs[k]
    coeffs.pop(-1)
    return coeffs

In [5]:
def laguerre_horner_deflacion(coeffs, z0, r=5, M=20):
    coefficients = coeffs[:]
    roots = []
    if len(coefficients) >= 8: 
        print('Aplicando el algoritmo de Laguerre-Horner se obtiene que el polinomio \n')
    else:
        print('Aplicando el algoritmo de Laguerre-Horner se obtiene que el polinomio', Polynomial(coefficients, 0), '\n')
    for i in range(1,len(coeffs)):
        z = laguerre_horner(coeffs, z0, 1e-6)
        z = round(z.real,4)+round(z.imag,4)*1j
        z = laguerre_horner(coeffs, z, 1e-8)
        z = round(z.real,r)+round(z.imag,r)*1j
        coeffs = deflacion(coeffs,z)
        roots.append(z)
        print('\xb7','tiene un cero en el punto z =', round(z.real,4)+round(z.imag,4)*1j,'\n')

In [6]:
#Ejemplo de uso
coeffs = [-2+8*1j, 0, 4/5, -1-8.5*1j, -1]
z0 = 1+0*1j
laguerre_horner_deflacion(coeffs, z0)

Aplicando el algoritmo de Laguerre-Horner se obtiene que el polinomio (-2+8j)*z^4 + 0.8*z^2 + (-1-8.5j)*z - 1 

· tiene un cero en el punto z = (1.0139-0.1281j) 

· tiene un cero en el punto z = (-0.014+0.1174j) 

· tiene un cero en el punto z = (-0.366+0.8676j) 

· tiene un cero en el punto z = (-0.634-0.8568j) 



In [7]:
def factorizacion(coeffs, z0, r=7, M=20):
    coefficients = coeffs[:]
    if coeffs[0] != 0:
        coeffs = [x /coeffs[0] for x in coeffs]
    else:
        return print('Error! El primer coeficiente es 0')
    roots = []
    if len(coefficients) >= 8: 
        print("La factorización del polinomio es \n")
    else:
        print("La factorización del polinomio", Polynomial(coefficients, 0),"es \n")
    for i in range(1,len(coeffs)):
        z = laguerre_horner(coeffs, z0, 1e-6)
        z = round(z.real,4)+round(z.imag,4)*1j
        z = laguerre_horner(coeffs, z, 1e-8)
        z = round(z.real,r)+round(z.imag,r)*1j
        coeffs = deflacion(coeffs,z)
        roots.append(z)
    return print(Factorization(roots,coefficients), "\n")

In [8]:
#Ejemplo de uso
coeffs = [-2+8*1j, 0, 4/5, -1-8.5*1j, -1]
z0 = 1+0*1j
factorizacion(coeffs, z0)

La factorización del polinomio (-2+8j)*z^4 + 0.8*z^2 + (-1-8.5j)*z - 1 es 

(-2+8*i)(z - 1.0139124 + 0.1281285*i)(z + 0.0139789 - 0.1174063*i)(z + 0.3659872 - 0.8675475*i)(z + 0.6339463 + 0.8568253*i) 



In [9]:
#Polinomio pérfido de Wilkinson (polinomio con raíces z=1,2,...,20)
coeffs = [1, -210, 20615, -1256850, 53327946,-1672280820,40171771630,-756111184500,11310276995381,-135585182899530,
          1307535010540395, -10142299865511450,63030812099294896,-311333643161390640, 1206647803780373360,-3599979517947607200,
          8037811822645051776, -12870931245150988800, 13803759753640704000, -8752948036761600000, 2432902008176640000]
z0 = 1+0*1j
laguerre_horner_deflacion(coeffs,z0)

Aplicando el algoritmo de Laguerre-Horner se obtiene que el polinomio 

· tiene un cero en el punto z = (1+0j) 

· tiene un cero en el punto z = (2+0j) 

· tiene un cero en el punto z = (3+0j) 

· tiene un cero en el punto z = (4+0j) 

· tiene un cero en el punto z = (5+0j) 

· tiene un cero en el punto z = (6+0j) 

· tiene un cero en el punto z = (7+0j) 

· tiene un cero en el punto z = (8+0j) 

· tiene un cero en el punto z = (9+0j) 

· tiene un cero en el punto z = (10+0j) 

· tiene un cero en el punto z = (11+0j) 

· tiene un cero en el punto z = (12+0j) 

· tiene un cero en el punto z = (13+0j) 

· tiene un cero en el punto z = (14+0j) 

· tiene un cero en el punto z = (15+0j) 

· tiene un cero en el punto z = (16+0j) 

· tiene un cero en el punto z = (17+0j) 

· tiene un cero en el punto z = (18+0j) 

· tiene un cero en el punto z = (19+0j) 

· tiene un cero en el punto z = (20+0j) 

